# Project : It is your job to predict the sales price for each house. For each Id in the test set, you must predict the value of the SalePrice variable. 

# Read dataset

In [45]:
from warnings import filterwarnings
filterwarnings("ignore")
import pandas as mt
train = mt.read_csv("C:/Users/SHREE/Desktop/Project/training_set.csv")

# Preview

In [46]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# Dealing with missing data

In [47]:
train.PoolQC = train.PoolQC.fillna("No_Pool")

In [48]:
train.Alley = train.Alley.fillna("No_alley")

In [49]:
from PM8 import replacer
replacer(train)

# Define our Y column and X column

In [50]:
Y = train[["SalePrice"]]
X = train.drop(labels=["SalePrice","Id"],axis=1)

# EDA


In [51]:
train.corr()['SalePrice'].sort_values().tail(11).index

Index(['YearRemodAdd', 'YearBuilt', 'TotRmsAbvGrd', 'FullBath', '1stFlrSF',
       'TotalBsmtSF', 'GarageArea', 'GarageCars', 'GrLivArea', 'OverallQual',
       'SalePrice'],
      dtype='object')

In [52]:
imp_con_cols = list(train.corr()['SalePrice'].sort_values().tail(11).index)
imp_con_cols.remove("SalePrice")

In [53]:
#imp_con_cols

In [54]:
cat = []
con = []
for i in X.columns:
    if(X[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

In [55]:
from PM8 import ANOVA
imp_cat_cols = []
for i in cat:
    pval = ANOVA(train,i,'SalePrice')
    #print("SalePrice vs",i,ANOVA(train,i,'SalePrice'))
    if(pval < 0.05):
        imp_cat_cols.append(i)

In [56]:
#imp_cat_cols

In [57]:
imp_cols = []
imp_cols.extend(imp_con_cols)
imp_cols.extend(imp_cat_cols)

# Preprocessing and splitting

In [58]:
import numpy as np
#X[imp_con_cols].apply(np.log)

# Deal with skew

In [59]:
for i in imp_con_cols:
    Q = []
    for j in X[i]:
        if(j != 0):
            Q.append(np.log(j))
        else:
            Q.append(0)
    X[i] = Q

In [60]:
X[imp_con_cols].skew()

YearRemodAdd   -0.511009
YearBuilt      -0.641144
TotRmsAbvGrd   -0.190746
FullBath       -0.084139
1stFlrSF        0.079157
TotalBsmtSF    -5.152705
GarageArea     -3.480590
GarageCars     -0.355098
GrLivArea      -0.006995
OverallQual    -0.929781
dtype: float64

In [61]:
from PM8_weekend import preprocessing
Xnew = preprocessing(X[imp_cols])

# OLS(Linear model)

In [62]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(Xnew,Y,test_size=0.2,random_state=21)


from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.911
Model:                            OLS   Adj. R-squared:                  0.891
Method:                 Least Squares   F-statistic:                     46.24
Date:                Wed, 01 Jun 2022   Prob (F-statistic):               0.00
Time:                        11:23:46   Log-Likelihood:                -13413.
No. Observations:                1168   AIC:                         2.725e+04
Df Residuals:                     956   BIC:                         2.832e+04
Df Model:                         211                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  1.946e+16   3.81e+16      0.510      0.610   -5.54e+16    9.43e+16
YearRemodAdd           3183.9230   1365.820      2.331      0.020     503.573    5864.274
YearBuilt              5744.7006   2727.879      2.106      0.035     391.379    1.11e+04
TotRmsAbvGrd            579.5829   1792.419      0.323      0.746   -2937.947    4097.113
FullBath               1846.9519   1461.091      1.264      0.207   -1020.365    4714.269
1stFlrSF               8052.2676   3149.100      2.557      0.011    1872.321    1.42e+04
TotalBsmtSF            2301.5763   1607.448      1.432      0.153    -852.957    5456.109
GarageArea              821.2220   1274.824      0.644      0.520   -1680.555    3322.999
GarageCars             3560.2512   1384.032      2.572      0.010     844.160    6276.343
GrLivArea              2.347e+04   3969.818      5.913      0.000    1.57e+04    3.13e+04
OverallQual            5598.8488   1683.462      3.326      0.001    2295.140    8902.557
MSZoning_C (all)       4.432e+15   8.68e+15      0.510      0.610   -1.26e+16    2.15e+16
MSZoning_FV            4.432e+15   8.68e+15      0.510      0.610   -1.26e+16    2.15e+16
MSZoning_RH            4.432e+15   8.68e+15      0.510      0.610   -1.26e+16    2.15e+16
MSZoning_RL            4.432e+15   8.68e+15      0.510      0.610   -1.26e+16    2.15e+16
MSZoning_RM            4.432e+15   8.68e+15      0.510      0.610   -1.26e+16    2.15e+16
Alley_Grvl            -5.035e+15   9.86e+15     -0.510      0.610   -2.44e+16    1.43e+16
Alley_No_alley        -5.035e+15   9.86e+15     -0.510      0.610   -2.44e+16    1.43e+16
Alley_Pave            -5.035e+15   9.86e+15     -0.510      0.610   -2.44e+16    1.43e+16
LotShape_IR1           1.112e+15   2.18e+15      0.510      0.610   -3.16e+15    5.39e+15
LotShape_IR2           1.112e+15   2.18e+15      0.510      0.610   -3.16e+15    5.39e+15
LotShape_IR3           1.112e+15   2.18e+15      0.510      0.610   -3.16e+15    5.39e+15
LotShape_Reg           1.112e+15   2.18e+15      0.510      0.610   -3.16e+15    5.39e+15
LandContour_Bnk        -3.14e+15   6.15e+15     -0.510      0.610   -1.52e+16    8.93e+15
LandContour_HLS        -3.14e+15   6.15e+15     -0.510      0.610   -1.52e+16    8.93e+15
LandContour_Low        -3.14e+15   6.15e+15     -0.510      0.610   -1.52e+16    8.93e+15
LandContour_Lvl        -3.14e+15   6.15e+15     -0.510      0.610   -1.52e+16    8.93e+15
LotConfig_Corner       2.813e+14   5.51e+14      0.510      0.610      -8e+14    1.36e+15
LotConfig_CulDSac      2.813e+14   5.51e+14      0.510      0.610      -8e+14    1.36e+15
LotConfig_FR2          2.813e+14   5.51e+14      0.510      0.610      -8e+14    1.36e+15
LotConfig_FR3          2.813e+14   5.51e+14      0.510      0.610      -8e+14    1.36e+15
LotConfig_Inside       

In [63]:
Q = mt.DataFrame(model.pvalues,columns=["Pval"])
col_to_drop = list(Q.sort_values(by=['Pval']).tail(1).index)

In [64]:
Q = mt.DataFrame(model.pvalues,columns=["Pval"])
col_to_drop = list(Q.sort_values(by=['Pval']).tail(1).index)

Xnew = Xnew.drop(labels=col_to_drop,axis=1)
xtrain,xtest,ytrain,ytest=train_test_split(Xnew,Y,test_size=0.2,random_state=21)
from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.911
Model:                            OLS   Adj. R-squared:                  0.891
Method:                 Least Squares   F-statistic:                     46.51
Date:                Wed, 01 Jun 2022   Prob (F-statistic):               0.00
Time:                        11:23:47   Log-Likelihood:                -13413.
No. Observations:                1168   AIC:                         2.725e+04
Df Residuals:                     957   BIC:                         2.832e+04
Df Model:                         210                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  2.158e+16   4.27e+16      0.506      0.613   -6.21e+16    1.05e+17
YearRemodAdd           3184.8534   1364.928      2.333      0.020     506.255    5863.452
YearBuilt              5757.2106   2727.257      2.111      0.035     405.117    1.11e+04
FullBath               1894.5322   1453.805      1.303      0.193    -958.482    4747.546
1stFlrSF               8045.8167   3147.180      2.557      0.011    1869.646    1.42e+04
TotalBsmtSF            2285.6695   1605.880      1.423      0.155    -865.784    5437.123
GarageArea              801.1915   1272.721      0.630      0.529   -1696.454    3298.837
GarageCars             3555.4002   1383.085      2.571      0.010     841.170    6269.630
GrLivArea              2.392e+04   3705.804      6.455      0.000    1.66e+04    3.12e+04
OverallQual            5597.9975   1682.521      3.327      0.001    2296.140    8899.855
MSZoning_C (all)         4.8e+15   9.49e+15      0.506      0.613   -1.38e+16    2.34e+16
MSZoning_FV              4.8e+15   9.49e+15      0.506      0.613   -1.38e+16    2.34e+16
MSZoning_RH              4.8e+15   9.49e+15      0.506      0.613   -1.38e+16    2.34e+16
MSZoning_RL              4.8e+15   9.49e+15      0.506      0.613   -1.38e+16    2.34e+16
MSZoning_RM              4.8e+15   9.49e+15      0.506      0.613   -1.38e+16    2.34e+16
Alley_Grvl            -4.243e+15   8.39e+15     -0.506      0.613   -2.07e+16    1.22e+16
Alley_No_alley        -4.243e+15   8.39e+15     -0.506      0.613   -2.07e+16    1.22e+16
Alley_Pave            -4.243e+15   8.39e+15     -0.506      0.613   -2.07e+16    1.22e+16
LotShape_IR1           9.383e+14   1.85e+15      0.506      0.613    -2.7e+15    4.58e+15
LotShape_IR2           9.383e+14   1.85e+15      0.506      0.613    -2.7e+15    4.58e+15
LotShape_IR3           9.383e+14   1.85e+15      0.506      0.613    -2.7e+15    4.58e+15
LotShape_Reg           9.383e+14   1.85e+15      0.506      0.613    -2.7e+15    4.58e+15
LandContour_Bnk       -2.954e+15   5.84e+15     -0.506      0.613   -1.44e+16     8.5e+15
LandContour_HLS       -2.954e+15   5.84e+15     -0.506      0.613   -1.44e+16     8.5e+15
LandContour_Low       -2.954e+15   5.84e+15     -0.506      0.613   -1.44e+16     8.5e+15
LandContour_Lvl       -2.954e+15   5.84e+15     -0.506      0.613   -1.44e+16     8.5e+15
LotConfig_Corner       4.347e+13   8.59e+13      0.506      0.613   -1.25e+14    2.12e+14
LotConfig_CulDSac      4.347e+13   8.59e+13      0.506      0.613   -1.25e+14    2.12e+14
LotConfig_FR2          4.347e+13   8.59e+13      0.506      0.613   -1.25e+14    2.12e+14
LotConfig_FR3          4.347e+13   8.59e+13      0.506      0.613   -1.25e+14    2.12e+14
LotConfig_Inside       4.347e+13   8.59e+13      0.506      0.613   -1.25e+14    2.12e+14
Neighborhood_Blmngtn   

# Create a model and test

In [65]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
model = lm.fit(xtrain,ytrain)
tr_pred = model.predict(xtrain)
ts_pred = model.predict(xtest)
from sklearn.metrics import mean_absolute_error
tr_err = mean_absolute_error(ytrain,tr_pred)
ts_err = mean_absolute_error(ytest,ts_pred)

In [66]:
tr_err

15973.5

In [67]:
ts_err

143149748288413.34

# Regularize

In [68]:
Q = list(Xnew.columns)
Q.extend(con)

In [69]:
duplicates = []
for i in Q:
    if(Q.count(i) > 1 and duplicates.count(i) == 0):
        duplicates.append(i)

In [70]:
Xnew[duplicates].corr()

,YearRemodAdd,YearBuilt,FullBath,1stFlrSF,TotalBsmtSF,GarageArea,GarageCars,GrLivArea,OverallQual
YearRemodAdd,1.000000,0.589813,0.451403,0.233819,0.205457,0.207472,0.453795,0.311515,0.536980
YearBuilt,0.589813,1.000000,0.490966,0.277025,0.198172,0.341475,0.556988,0.232931,0.553431
FullBath,0.451403,0.490966,1.000000,0.373942,0.094538,0.228520,0.537545,0.652849,0.531549
1stFlrSF,0.233819,0.277025,0.373942,1.000000,0.288502,0.286945,0.441902,0.545984,0.431252
TotalBsmtSF,0.205457,0.198172,0.094538,0.288502,1.000000,0.177266,0.182415,0.206764,0.364240
GarageArea,0.207472,0.341475,0.228520,0.286945,0.177266,1.000000,0.530889,0.298299,0.397636
GarageCars,0.453795,0.556988,0.537545,0.441902,0.182415,0.530889,1.000000,0.496139,0.556868
GrLivArea,0.311515,0.232931,0.652849,0.545984,0.206764,0.298299,0.496139,1.000000,0.601148
OverallQual,0.536980,0.553431,0.531549,0.431252,0.364240,0.397636,0.556868,0.601148,1.000000


# GridSearchCV

In [71]:
from sklearn.model_selection import GridSearchCV

In [72]:
grid = []
w = 0.9
for i in range(0,10,1):
    grid.append(w)
    w = w + 0.001

# Lasso

In [73]:
from sklearn.linear_model import Lasso
ls = Lasso()
tg = {"alpha":grid}
cv = GridSearchCV(ls,tg,scoring="neg_mean_absolute_error",cv=3)
cvmodel = cv.fit(xtrain,ytrain)
cvmodel.best_params_

{'alpha': 0.909}

In [75]:
ls = Lasso(alpha=0.909)
model = ls.fit(Xnew,Y)

# Read data

In [76]:
X2 = mt.read_csv("C:/Users/SHREE/Desktop/Project/testing_set (1).csv")

# Preview

In [77]:
X2

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


# Missing data

In [78]:
replacer(X2)

# Preprocessing

In [79]:
X2new = preprocessing(X2)

In [80]:
#X2new[list(Xnew.columns)]

In [81]:
Q = ['Exterior1st_ImStucc', 'Electrical_Mix', 'Condition2_RRAn', 'RoofMatl_ClyTile', 'HouseStyle_2.5Fin', 'GarageQual_Ex', 'Condition2_RRAe', 'RoofMatl_Membran', 'RoofMatl_Roll', 'Alley_No_alley', 'Exterior2nd_Other', 'Heating_Floor', 'Heating_OthW', 'PoolQC_No_Pool', 'RoofMatl_Metal', 'Condition2_RRNn', 'PoolQC_Fa']

In [82]:
for i in Q:
    X2new[i]=0

In [83]:
X2new.shape

(1459, 288)

In [84]:
Xnew.shape

(1460, 252)

In [85]:
Q = ['HouseStyle_2.5Fin', 'PoolQC_No_Pool', 'Alley_No_alley','Exterior1st_Stone']
for i in Q:
    X2new[i]=0

In [86]:
finalX = X2new[Xnew.columns]

# Prediction 

In [87]:
pred = model.predict(finalX)

In [42]:
finalDF = X2[["Id"]]
finalDF['SalePrice']=pred

In [90]:
finalDF

,Id,SalePrice
0,1461,258495.198866
1,1462,314452.758409
2,1463,338982.739990
3,1464,347993.297928
4,1465,366785.804928
...,...,...
1454,2915,234892.811041
1455,2916,251379.596765
1456,2917,307528.176780
1457,2918,275340.912902
